In [ ]:
#라이브러리 호출
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from wordcloud import WordCloud
okt = Okt()
%matplotlib inline


In [ ]:
#한글폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
#url변수
url_infp = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-infp"
url_infj = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-infj"
url_enfj = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-enfj"
url_enfp = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-enfp"
url_istp = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-istp"
url_isfp = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-isfp"
url_estp = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-estp"
url_esfp = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-esfp"
url_istj = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-istj"
url_isfj = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-isfj"
url_estj = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-estj"
url_esfj = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-esfj"
url_intj = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-intj"
url_intp = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-intp"
url_entj = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-entj"
url_entp = "https://www.16personalities.com/ko/%EC%84%B1%EA%B2%A9%EC%9C%A0%ED%98%95-entp"

def web_scrapping(url):
    #웹스크래핑
    res =requests.get(url)
    res.raise_for_status() #웹페이지에 접근가능한지 체크
    soup = BeautifulSoup(res.text, "lxml") #lxml = 파서
    text= soup.find(attrs= {"class":"main description"}).get_text() #Xpath 경로를 활용해 스크래핑
    print("원본 텍스트: "+ text)

    #텍스트 클렌징
    cleansed_text = re.sub('[^\s\w]', ' ', text)
    cleansed_text = re.sub('\s+', ' ', cleansed_text)
    print("클렌징된 텍스트: ", cleansed_text, "\n")

    #품사태깅
    pos_word_list = okt.pos(cleansed_text, stem=True)
    pos_list = ['Adjective'] #형용사가 유의미한 정보를 잘 나타내주는 것 같아서 동사,명사는 제외
    pos_filtered_list=[]
    for word, pos in pos_word_list:
        if pos in pos_list:
            pos_filtered_list.append(word)
    print("품사태깅된 텍스트: ", pos_filtered_list, "\n")
    
    #불용어 제거
    new_pos_filtered_list = []
    stop_word_list = ["있다", "이다", "없다","아니다", "중요하다","어떠하다","좋다","이러하다","그렇다","좋다", "필요하다","좋아하다","같다","많다","적다","이렇다","이러하다"]
    
    for i in pos_filtered_list:
        if i not in stop_word_list:
            new_pos_filtered_list.append(i)
    print("불용어 제거된 리스트: ", new_pos_filtered_list, '\n')

    #워드 클라우드 생성
    c = Counter(new_pos_filtered_list)
    print("단어별 빈도수: ", c.most_common(100), "\n")

    #워드 클라우드 시각화
    word_dict = dict(c.most_common(100))

    wc = WordCloud(font_path = 'NanumGothic.ttf',
               background_color ='white',
               width= 3000, height=2000,
               min_font_size= 10)

    cloud = wc.generate_from_frequencies(word_dict)
    #워드 클라우드 출력
    plt.imshow(cloud)
    plt.axis('off')
    plt.show()

#함수 실행
web_scrapping(url_isfp)
